In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

/data/jiacheng/miniconda3/envs/common/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [3]:
INPUT_DIR = "/home/ljc/data/graphrag/alltest/basedataset/test1/output/20240906-163058/artifacts"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [4]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 698


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,top_level_node_id,x,y
0,0,UNITED STATES OF AMERICA,GEO,"The United States of America, commonly known a...",cc0dbbd456ec94f96f25793657991734,2,22,0,b45241d70f0e43fca764df95b2b81f77,22.0,"[-0.02083694003522396, -0.15422822535037994, 0...",b45241d70f0e43fca764df95b2b81f77,3.038575,0.381777
1,0,"WASHINGTON, D.C.",GEO,"Washington, D.C. is the capital city and feder...","aa86232b2bed138127746cc93ab4c0bc,cc0dbbd456ec9...",3,21,1,4119fd06010c494caa07f439b333f4c5,21.0,"[0.024507159367203712, -0.10291264951229095, 0...",4119fd06010c494caa07f439b333f4c5,3.875553,-1.794526
2,0,THIRTEEN COLONIES,GEO,The Thirteen Colonies were the first British s...,cc0dbbd456ec94f96f25793657991734,2,2,2,d3835bf3dda84ead99deadbeac5d0d7d,2.0,"[-0.0007702750153839588, -0.10679806023836136,...",d3835bf3dda84ead99deadbeac5d0d7d,3.811445,0.534127
3,0,CONFEDERATE STATES OF AMERICA,GEO,The Confederate States of America were a group...,cc0dbbd456ec94f96f25793657991734,2,1,3,077d2820ae1845bcbb1803379a3d1eae,1.0,"[-0.01923474296927452, -0.07166963815689087, 0...",077d2820ae1845bcbb1803379a3d1eae,4.996119,2.853426
4,0,UNION,GEO,The Union refers to the northern states during...,cc0dbbd456ec94f96f25793657991734,2,1,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,1.0,"[-0.015743521973490715, -0.07381941378116608, ...",3671ea0dd4e84c1a9b02c5ab2c8f4bac,4.881209,2.672398


#### Read relationships

In [5]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 359


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,UNITED STATES OF AMERICA,"WASHINGTON, D.C.",9.0,"Washington, D.C. is the federal capital distri...",[cc0dbbd456ec94f96f25793657991734],b45ef27279c043269b23b894461d7d8c,0,22,21,43
1,UNITED STATES OF AMERICA,THIRTEEN COLONIES,8.0,The Thirteen Colonies were the original Britis...,[cc0dbbd456ec94f96f25793657991734],10983a248cc448c59c94df4d1d0898f0,1,22,2,24
2,UNITED STATES OF AMERICA,CONFEDERATE STATES OF AMERICA,7.0,The Confederate States of America seceded from...,[cc0dbbd456ec94f96f25793657991734],e2ec7d3cdbeb4dd086ae6eb399332363,2,22,1,23
3,UNITED STATES OF AMERICA,UNION,7.0,The Union was the term used for the northern s...,[cc0dbbd456ec94f96f25793657991734],67f10971666240ea930f3b875aabdc1a,3,22,1,23
4,UNITED STATES OF AMERICA,PEARL HARBOR,8.0,The attack on Pearl Harbor led to the United S...,[cc0dbbd456ec94f96f25793657991734],8b95083939ad4771b57a97c2d5805f36,4,22,1,23


In [6]:
# NOTE: covariates are turned off by default, because they generally need prompt tuning to be valuable
# Please see the GRAPHRAG_CLAIM_* settings
# covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

# claims = read_indexer_covariates(covariate_df)

# print(f"Claim records: {len(claims)}")
# covariates = {"claims": claims}

#### Read community reports

In [7]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
print(reports)

report_df.head()

Report records: 33
[CommunityReport(id='11', short_id='11', title='Nice and Its Historical and Cultural Significance', community_id='11', summary="The community centers around Nice, a city on the French Riviera, known for its historical and cultural significance. It has been a winter resort for European aristocracy and is home to many notable figures and events. The city's relationships with entities like UNESCO, Monaco, and various historical figures highlight its importance.", full_content="# Nice and Its Historical and Cultural Significance\n\nThe community centers around Nice, a city on the French Riviera, known for its historical and cultural significance. It has been a winter resort for European aristocracy and is home to many notable figures and events. The city's relationships with entities like UNESCO, Monaco, and various historical figures highlight its importance.\n\n## Nice as a UNESCO World Heritage Site\n\nNice was proclaimed a UNESCO World Heritage Site in 2021, undersco

,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,11,# Nice and Its Historical and Cultural Signifi...,1,8.5,Nice and Its Historical and Cultural Significance,The impact severity rating is high due to Nice...,"The community centers around Nice, a city on t...",[{'explanation': 'Nice was proclaimed a UNESCO...,"{\n ""title"": ""Nice and Its Historical and C...",c3113814-5f00-48c8-9dbb-e8d91acbb825
1,12,# Marseille and Its Metropolitan Influence\n\n...,1,8.5,Marseille and Its Metropolitan Influence,The impact severity rating is high due to Mars...,"The community centers around Marseille, a majo...",[{'explanation': 'Marseille is a critical econ...,"{\n ""title"": ""Marseille and Its Metropolita...",2cfa6ada-a1de-4898-84c1-55a1879f4fed
2,13,# North America and Bering Land Bridge\n\nThe ...,1,7.0,North America and Bering Land Bridge,The impact severity rating is high due to the ...,The community is centered around North America...,[{'explanation': 'North America is a key entit...,"{\n ""title"": ""North America and Bering Land...",e296bedd-a2d7-4b8d-a505-6b9c68e8b2d0
3,14,# Cold War and the Iron Curtain\n\nThe communi...,1,9.0,Cold War and the Iron Curtain,The impact severity rating is high due to the ...,"The community is centered around the Cold War,...",[{'explanation': 'The Cold War was a period of...,"{\n ""title"": ""Cold War and the Iron Curtain...",0f587d46-0a92-4274-8e49-0fe1e6f16450
4,15,# Thirteen Colonies and Virginia\n\nThe commun...,1,8.5,Thirteen Colonies and Virginia,The impact severity rating is high due to the ...,The community is centered around the historica...,[{'explanation': 'The Thirteen Colonies were t...,"{\n ""title"": ""Thirteen Colonies and Virgini...",9065da63-8dab-42ba-bed9-550ffb2caac5


#### Read text units

In [8]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 16


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,cc0dbbd456ec94f96f25793657991734,"The United States of America (USA or U.S.A.), ...",669,[0134252ef584e1c209f4b4937c1d7a4f],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b45ef27279c043269b23b894461d7d8c, 10983a248cc..."
1,b324dfcc8a3699591c60646ccaf6e4b0,Nice (/niːs/ NEESS; French pronunciation: [nis...,1035,[10dbe8991bb9a7ab1dda37e9ccbedd78],"[e657b5121ff8456b9a610cfaead8e0cb, bf4e255cdac...","[d6d2b5862ddc4c4d87deee3423506817, 47d588d26e2..."
2,c796265ad38bbdc79dc17b2d0aba7e9e,"New York, often called New York City[b] or NYC...",746,[147a4bae12320a91e5843421457c4c0e],"[9646481f66ce4fd2b08c2eddda42fc82, 4f3c97517f7...","[b4e4fa2e3dfc46e68d532d659b18d17d, 48cd97f2297..."
3,e201254a65d111262b3361be30dbaa31,The European Union (EU) is a supranational pol...,621,[45118b491c6d36b4a415322d2c444105],"[07b2425216bd4f0aa4e079827cb48ef5, 7c49f2710e8...","[a701c349eb7142d48ba7efad89caf9d2, 49f771e31a0..."
4,7f6347c263ce28650c650f3f050a4f70,Paris (French pronunciation: [paʁi] ⓘ) is the ...,825,[474b35b12bfb44f1235cd856dcae3b43],"[ef32c4b208d041cc856f6837915dc1b0, 07b2425216b...","[fd139ac75b0e4777ab67b7423eaaa37f, e5d40a1b17f..."


In [9]:
api_key = os.environ["OPENAI_API_KEY"]
llm_model = 'gpt-4o-2024-08-06'
embedding_model = 'text-embedding-3-small'

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

### Create local search context builder

In [10]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None
    # covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [11]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [12]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [13]:
question = "Is Marc Chagall associated with Nice? Answer with a yes or no. And then explain why."
result = await search_engine.asearch(question)
print(result.response)

Yes, Marc Chagall is associated with Nice.

### Association with Nice

Marc Chagall, a notable painter, has a significant connection to the city of Nice. His work is commemorated in the city's museums, which highlights his influence and contribution to the cultural landscape of Nice [Data: Entities (33); Relationships (73)]. The presence of Chagall's art in Nice's museums not only enriches the city's cultural heritage but also attracts art enthusiasts from around the world, further cementing his association with the city.

### Cultural Impact

The inclusion of Chagall's work in Nice's museums is part of a broader cultural tapestry that includes other renowned artists such as Henri Matisse and Niki de Saint Phalle, whose works are also celebrated in the city [Data: Entities (34, 35); Relationships (74, 75)]. This collective artistic presence underscores Nice's role as a hub for artistic expression and its historical significance as a destination for artists and writers. Chagall's associ

#### Inspecting the context data used to generate the response

In [14]:
result.context_data["entities"]

,id,entity,description,number of relationships,in_context
0,33,MARC CHAGALL,Marc Chagall was a notable painter whose work ...,1,True
1,34,HENRI MATISSE,Henri Matisse was a notable painter whose work...,1,True
2,35,NIKI DE SAINT PHALLE,Niki de Saint Phalle was a notable painter who...,1,True
3,25,NICE,Nice is a city in the Alpes-Maritimes departme...,25,True
4,36,ARMAN,Arman was a notable painter whose work is comm...,1,True
5,42,CIMETIÈRE DU CHÂTEAU,A cemetery in Nice where many celebrated indiv...,1,True
6,26,FRENCH RIVIERA,The southeastern coast of France on the Medite...,1,True
7,41,"RUSSIAN ORTHODOX CEMETERY, NICE",A cemetery in Nice where notable Russian figur...,1,True
8,39,ANTON CHEKHOV,Anton Chekhov was a Russian writer who complet...,1,True
9,29,"PRINCE ARTHUR, DUKE OF CONNAUGHT",Prince Arthur inaugurated the Promenade des An...,1,True


In [15]:
result.context_data["relationships"]

,id,source,target,description,weight,rank,links,in_context
0,88,NICE,PROVENCE-ALPES-CÔTE D'AZUR,Nice is the second-largest city in the Provenc...,8.0,27,1,True
1,68,NICE,FRENCH RIVIERA,Nice is located on the French Riviera,9.0,26,1,True
2,69,NICE,MENICA RONDELLY,Menica Rondelly wrote the unofficial anthem of...,7.0,26,1,True
3,73,NICE,MARC CHAGALL,Marc Chagall's work is commemorated in Nice's ...,7.0,26,3,True
4,74,NICE,HENRI MATISSE,Henri Matisse's work is commemorated in Nice's...,7.0,26,3,True
5,75,NICE,NIKI DE SAINT PHALLE,Niki de Saint Phalle's work is commemorated in...,7.0,26,3,True
6,76,NICE,ARMAN,Arman's work is commemorated in Nice's museums,7.0,26,1,True
7,77,NICE,FRANK HARRIS,Frank Harris wrote several books in Nice,6.0,26,1,True
8,79,NICE,ANTON CHEKHOV,"Anton Chekhov completed his play ""Three Sister...",7.0,26,1,True
9,81,NICE,"RUSSIAN ORTHODOX CEMETERY, NICE",The Russian Orthodox Cemetery is located in Nice,6.0,26,1,True


In [16]:
result.context_data["reports"]

,id,title,content
0,11,Nice and Its Historical and Cultural Significance,# Nice and Its Historical and Cultural Signifi...
1,27,"Paris: A Global Hub of Culture, Commerce, and ...","# Paris: A Global Hub of Culture, Commerce, an..."
2,11,Nice and Its Historical and Cultural Significance,# Nice and Its Historical and Cultural Signifi...
3,27,"Paris: A Global Hub of Culture, Commerce, and ...","# Paris: A Global Hub of Culture, Commerce, an..."


In [17]:
result.context_data["sources"]

,id,text
0,1,Nice (/niːs/ NEESS; French pronunciation: [nis...
1,4,Paris (French pronunciation: [paʁi] ⓘ) is the ...
2,11,Marseille or Marseilles (French: Marseille; Pr...


In [18]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [19]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [20]:
question_history = [
    "Tell me about Paris",
    "What happens in Paris 2024?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)

['- What major international events are scheduled to take place in Paris in 2024?', '- How is Paris preparing for the 2024 Summer Olympics?', "- What impact will the 2024 Summer Olympics have on Paris's economy and infrastructure?", '- Which sports will be featured in the 2024 Summer Olympics in Paris?', '- How will the 2024 Summer Olympics influence tourism in Paris?']
